## Effects of MAM Pacific SST and uwind on JJA Borneo rainfall.

Here we investigate the effects of sea surface temperature over the eastern North Pacific (MAM SST) and zonal wind over the western equatorial Pacific in the boreal spring months (MAM uwind) on mean precipitaion over Indonesian Borneo (BorneoP) in the boreal summer months (JJA) in the near future (2021 - 2060) and far future (2061 - 2100), using an ensemble of CMIP6 models (based on SSP245 and SSP585 scenarios).

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

import os

import iris
import iris.coord_categorisation
import datetime
from iris.time import PartialDateTime

import iris.quickplot as qplt

from iris.pandas import as_cube, as_series, as_data_frame
import cf_units


from scipy import signal
import scipy.stats as stats

import statsmodels.api as sm
import pandas as pd

import copy

import sklearn.linear_model as ML

DATA_PATH = '/CMIP6/'


# Load data JJA means

In [12]:
# Models with uwind outputs

models = ['ACCESS-CM', 'ACCESS-ESM', 'CanESM',
          'CNRM-CM', 'CNRM-ESM',  'GFDL', 
          'HadGEM3-GC31-LL', 'INM-CM4', 'INM-CM5', 
          'IPSL', 'MIROC', 'MPI-ESM1-2-LR', 'MRI', 'NESM', 
          'UKESM', 'CMCC-CM', 'CMCC-ESM', 'KIOST']
scenarios = ['ssp245_nf', 'ssp245_ff', 'ssp585_nf', 'ssp585_ff']

for member in models[0:]:
    for scenario in scenarios[0:]:
        print(member)
        print (scenario)
        precip_Borneo = iris.load_cube(DATA_PATH + 'pr/Borneo_precip_mean_' + member + '_' + scenario + '.nc', 'precipitation_flux')
        pacific_SST          = iris.load_cube(DATA_PATH + "sst/Pacific_SST_" + member + '_' + scenario + '.nc', 'sea_surface_temperature')
        pacific_wind = iris.load_cube(DATA_PATH + 'uwind/Pacific_wind_' + member + '_' + scenario + '.nc', 'eastward_wind')
        print(np.shape(pacific_SST),np.shape(pacific_wind), np.shape (precip_Borneo))
    
        # Standardise and detrend
        PacificSST_trend = pacific_SST - signal.detrend(pacific_SST.data)
        PacificSST_det = pacific_SST  - PacificSST_trend
        PACIFIC_SST = (PacificSST_det - np.mean(PacificSST_det.data))/np.std(PacificSST_det.data)

        PacificWind_trend = pacific_wind - signal.detrend(pacific_wind.data)
        PacificWind_det = pacific_wind  - PacificWind_trend
        PACIFIC_WIND = (PacificWind_det - np.mean(PacificWind_det.data))/np.std(PacificWind_det.data)

        pBorneo_trend = precip_Borneo - signal.detrend(precip_Borneo.data)
        pBorneo_det = precip_Borneo  - pBorneo_trend
        PBORNEO = (pBorneo_det - np.mean(pBorneo_det.data))/np.std(pBorneo_det.data)

        # Effect of MAM SST on MAM uwind
        Y = PACIFIC_WIND.data
        X = np.stack([PACIFIC_SST.data]).T
        model = sm.OLS(Y,X)
        results = model.fit()
        print(results.summary())
        
        # Effect of MAM SST on JJA BorneoP
        Y = PBORNEO.data
        X = np.stack([PACIFIC_SST.data]).T
        model = sm.OLS(Y,X)
        results = model.fit()
        print(results.summary())
        
        # Effect of MAM uwind on JJA BorneoP
        Y = PBORNEO.data
        X = np.stack([PACIFIC_WIND.data]).T
        model = sm.OLS(Y,X)
        results = model.fit()
        print(results.summary())
        
        # Effect of MAM SST and MAM uwind on JJA BorneoP
        Y = PBORNEO.data
        X = np.stack([PACIFIC_SST.data, PACIFIC_WIND.data]).T
        model = sm.OLS(Y,X)
        results = model.fit()
        print(results.summary())
        print("effect of Pacific SST on Dry Days=", round(results.params[0],2))
        print("effect of Pacific wind on Dry Days =", round(results.params[1],2))

(40,) (40,) (40,)
ACCESS-CM
ssp245_nf
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.368
Model:                            OLS   Adj. R-squared (uncentered):              0.352
Method:                 Least Squares   F-statistic:                              22.71
Date:                Tue, 03 Oct 2023   Prob (F-statistic):                    2.62e-05
Time:                        11:43:39   Log-Likelihood:                         -47.581
No. Observations:                  40   AIC:                                      97.16
Df Residuals:                      39   BIC:                                      98.85
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
---

(40,) (40,) (40,)
ACCESS-CM
ssp585_nf
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.289
Model:                            OLS   Adj. R-squared (uncentered):              0.271
Method:                 Least Squares   F-statistic:                              15.85
Date:                Tue, 03 Oct 2023   Prob (F-statistic):                    0.000290
Time:                        11:43:40   Log-Likelihood:                         -49.938
No. Observations:                  40   AIC:                                      101.9
Df Residuals:                      39   BIC:                                      103.6
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
---

(40,) (40,) (40,)
CanESM
ssp245_nf
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.279
Model:                            OLS   Adj. R-squared (uncentered):              0.260
Method:                 Least Squares   F-statistic:                              15.09
Date:                Tue, 03 Oct 2023   Prob (F-statistic):                    0.000387
Time:                        11:43:40   Log-Likelihood:                         -50.217
No. Observations:                  40   AIC:                                      102.4
Df Residuals:                      39   BIC:                                      104.1
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
------

(40,) (40,) (40,)
CanESM
ssp585_nf
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.157
Model:                            OLS   Adj. R-squared (uncentered):              0.135
Method:                 Least Squares   F-statistic:                              7.243
Date:                Tue, 03 Oct 2023   Prob (F-statistic):                      0.0104
Time:                        11:43:41   Log-Likelihood:                         -53.351
No. Observations:                  40   AIC:                                      108.7
Df Residuals:                      39   BIC:                                      110.4
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
------

(40,) (40,) (40,)
FGOALS-f3
ssp245_nf
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.194
Model:                            OLS   Adj. R-squared (uncentered):              0.173
Method:                 Least Squares   F-statistic:                              9.366
Date:                Tue, 03 Oct 2023   Prob (F-statistic):                     0.00399
Time:                        11:43:41   Log-Likelihood:                         -52.453
No. Observations:                  40   AIC:                                      106.9
Df Residuals:                      39   BIC:                                      108.6
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
---

...

(40,) (40,) (40,)
INM-CM4
ssp585_ff
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.011
Model:                            OLS   Adj. R-squared (uncentered):             -0.014
Method:                 Least Squares   F-statistic:                             0.4455
Date:                Tue, 03 Oct 2023   Prob (F-statistic):                       0.508
Time:                        11:43:52   Log-Likelihood:                         -56.530
No. Observations:                  40   AIC:                                      115.1
Df Residuals:                      39   BIC:                                      116.7
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----

(40,) (40,) (40,)
INM-CM5
ssp245_ff
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.013
Model:                            OLS   Adj. R-squared (uncentered):             -0.012
Method:                 Least Squares   F-statistic:                             0.5273
Date:                Tue, 03 Oct 2023   Prob (F-statistic):                       0.472
Time:                        11:43:52   Log-Likelihood:                         -56.489
No. Observations:                  40   AIC:                                      115.0
Df Residuals:                      39   BIC:                                      116.7
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----

(40,) (40,) (40,)
INM-CM5
ssp585_ff
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.156
Model:                            OLS   Adj. R-squared (uncentered):              0.135
Method:                 Least Squares   F-statistic:                              7.228
Date:                Tue, 03 Oct 2023   Prob (F-statistic):                      0.0105
Time:                        11:43:53   Log-Likelihood:                         -53.357
No. Observations:                  40   AIC:                                      108.7
Df Residuals:                      39   BIC:                                      110.4
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----

(40,) (40,) (40,)
IPSL
ssp245_ff
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.010
Model:                            OLS   Adj. R-squared (uncentered):             -0.015
Method:                 Least Squares   F-statistic:                             0.3958
Date:                Tue, 03 Oct 2023   Prob (F-statistic):                       0.533
Time:                        11:43:53   Log-Likelihood:                         -56.556
No. Observations:                  40   AIC:                                      115.1
Df Residuals:                      39   BIC:                                      116.8
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
--------

(40,) (40,) (40,)
IPSL
ssp585_ff
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.101
Model:                            OLS   Adj. R-squared (uncentered):              0.078
Method:                 Least Squares   F-statistic:                              4.398
Date:                Tue, 03 Oct 2023   Prob (F-statistic):                      0.0425
Time:                        11:43:54   Log-Likelihood:                         -54.620
No. Observations:                  40   AIC:                                      111.2
Df Residuals:                      39   BIC:                                      112.9
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
--------

In [15]:
# Models without uwind outputs

models = ['FGOALS-g3', 'NorESM2-LM', 'NorESM2-MM', 'TaiESM']
scenarios = ['ssp245_nf', 'ssp245_ff', 'ssp585_nf', 'ssp585_ff']

for member in models[0:]:
    for scenario in scenarios[0:]:
        precip_Borneo = iris.load_cube(DATA_PATH + 'pr/Borneo_precip_mean_' + member + '_' + scenario + '.nc', 'precipitation_flux')
        pacific_SST          = iris.load_cube(DATA_PATH + "sst/Pacific_SST_" + member + '_' + scenario + '.nc', 'sea_surface_temperature')
        print(np.shape(pacific_SST),np.shape(precip_Borneo))
    
        # Standardize and detrend
        PacificSST_trend = pacific_SST - signal.detrend(pacific_SST.data)
        PacificSST_det = pacific_SST  - PacificSST_trend
        PACIFIC_SST = (PacificSST_det - np.mean(PacificSST_det.data))/np.std(PacificSST_det.data)

        pBorneo_trend = precip_Borneo - signal.detrend(precip_Borneo.data)
        pBorneo_det = precip_Borneo  - pBorneo_trend
        PBORNEO = (pBorneo_det - np.mean(pBorneo_det.data))/np.std(pBorneo_det.data)
    
        print(member)
        print (scenario)
        
        # Effect of MAM SST on JJA BorneoP
        Y = PBORNEO.data
        X = np.stack([PACIFIC_SST.data]).T
        model = sm.OLS(Y,X)
        results = model.fit()
        print(results.summary())

(40,) (40,)
FGOALS-g3
ssp245_nf
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.152
Model:                            OLS   Adj. R-squared (uncentered):              0.130
Method:                 Least Squares   F-statistic:                              6.995
Date:                Tue, 03 Oct 2023   Prob (F-statistic):                      0.0117
Time:                        11:51:11   Log-Likelihood:                         -53.458
No. Observations:                  40   AIC:                                      108.9
Df Residuals:                      39   BIC:                                      110.6
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
---------

(40,) (40,)
NorESM2-LM
ssp585_nf
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.020
Model:                            OLS   Adj. R-squared (uncentered):             -0.006
Method:                 Least Squares   F-statistic:                             0.7804
Date:                Tue, 03 Oct 2023   Prob (F-statistic):                       0.382
Time:                        11:51:12   Log-Likelihood:                         -56.361
No. Observations:                  40   AIC:                                      114.7
Df Residuals:                      39   BIC:                                      116.4
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
--------